In [1]:
from collections import defaultdict
from functools import wraps, lru_cache

fun_call_time = defaultdict(int)

def get_call_time(f):
    @wraps(f)  # 保留原有函数名称
    def wrap(n):
        fun_call_time[f.__name__, n] += 1  # 函数出现的次数加一
        return f(n)
    return wrap

def memo(f):
    already_computed = {}  # 记录已经计算过的数据
    @wraps(f)
    def warp(n):
        if n in already_computed:
            return already_computed[n]
        else:
            already_computed[n] = f(n)
            return already_computed[n]
    return warp


solustion = {}
@memo
@get_call_time 
def r(n):
    max_price, max_split =  max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key=lambda x: x[0]
    )
    solustion[n] = (n - max_split, max_split)
    return max_price

def parse_solution(n):
    left, right = solustion[n]
    if right == 0:
        return [left]
    return parse_solution(left) + parse_solution(right)

original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]  # 定义价格
price = defaultdict(int)
for i, p in enumerate(original_price): 
    price[i + 1] = p

r(30)
parse_solution(30)


[11, 11, 6, 2]

In [2]:
solution = {}
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  # string 1 add tail of string2
    ]
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation 
    return min_distance

def edit_parse_solution(string1, string2):
    result = {}
    for i in range(len(string1)):
        result[(string1,string2)] = solution[(string1,string2)]
        string1, string2 = string1[0:-1], string2[0:-1]
    return sorted(result.items(),key=lambda x:x[0],reverse=False)

In [3]:
result = edit_distance('abddgf', 'abcdef')
print(result)
edit_parse_solution('abddgf','abcdef')

2


[(('a', 'a'), ''),
 (('ab', 'ab'), ''),
 (('abd', 'abc'), 'SUB d => c'),
 (('abdd', 'abcd'), ''),
 (('abddg', 'abcde'), 'SUB g => e'),
 (('abddgf', 'abcdef'), '')]